In [ ]:
# Generate a set of points and corresponding hyperplanes
num_points = 50
points = np.random.rand(num_points, 2)
hyperplanes = [find_hyperplane_parameters_regression(simple_network, point) for point in points]


# Initialize the model and define a function to use it with NumPy arrays
model = SimpleNN()

def model_function(x):
    return model(torch.tensor(x, dtype=torch.float32))


# Calculate the radii for each point
radii = calculate_radii(points, hyperplanes, simple_network)

# Perform gradient descent optimization
initial_weights = np.ones(len(points)) / len(points)
learning_rate = 4e-2
num_iterations = 400
optimized_weights, losses = gradient_descent(points, radii, hyperplanes, simple_network, initial_weights, learning_rate, num_iterations)

# Plot the convergence
plt.plot(np.array(losses))
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Convergence of Gradient Descent")
plt.show()

# Validate the final resulting weights
validation_points = np.random.rand(50, 2)
for x in validation_points:
    model_output = model_function(x)
    optimized_output = sum([optimized_weights[i] * (np.dot(hyperplanes[i][0], x - points[i]) + hyperplanes[i][1]) if np.linalg.norm(x - points[i]) <= radii[i] else 0 for i in range(len(points))])
    print(f"Model output: {model_output}, Optimized output: {optimized_output}")


# Create a validation dataset
num_validation_points = 100
validation_points = np.random.rand(num_validation_points, 2)

# Compute the mean squared error (MSE) between the model output and the optimized output
mse = 0
for x in validation_points:
    model_output = model_function(x)
    optimized_output = sum([optimized_weights[i] * (np.dot(hyperplanes[i][0], x - points[i]) + hyperplanes[i][1]) if np.linalg.norm(x - points[i]) <= radii[i] else 0 for i in range(len(points))])
    mse += (model_output - optimized_output) ** 2

mse /= num_validation_points
print(f"Mean squared error (MSE) between model output and optimized output: {mse}")


visualize_surface(simple_network)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a grid of points in the input space
grid_size = 50
x_range = np.linspace(0, 1, grid_size)
y_range = np.linspace(0, 1, grid_size)
X, Y = np.meshgrid(x_range, y_range)

# Compute the model output and optimized output for each point in the grid
model_output_grid = np.zeros((grid_size, grid_size))
optimized_output_grid = np.zeros((grid_size, grid_size))

for i, x in enumerate(x_range):
    for j, y in enumerate(y_range):
        point = np.array([x, y])
        model_output_grid[i, j] = model_function(point)
        optimized_output_grid[i, j] = sum([optimized_weights[k] * (np.dot(hyperplanes[k][0], point - points[k]) + hyperplanes[k][1]) if np.linalg.norm(point - points[k]) <= radii[k] else 0 for k in range(len(points))])

# Create a 3D plot of the model output and optimized output
fig = plt.figure(figsize=(16, 8))

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, model_output_grid, cmap='viridis', alpha=0.8)
ax1.set_title("Model Output")
ax1.set_xlabel("X-axis")
ax1.set_ylabel("Y-axis")
ax1.set_zlabel("Z-axis")

ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X, Y, optimized_output_grid, cmap='viridis', alpha=0.8)
ax2.set_title("Optimized Output")
ax2.set_xlabel("X-axis")
ax2.set_ylabel("Y-axis")
ax2.set_zlabel("Z-axis")

plt.show()




# Parameters for gradient descent
alpha = 0.01
num_iterations = 800
lambda_lasso = 1

# Optimize the objective function with Lasso regularization using gradient descent
optimized_weights_with_lasso = np.ones(len(points)) / len(points)
for _ in tqdm.tqdm(range(num_iterations)):
    grad = gradient_with_lasso(optimized_weights_with_lasso, points, hyperplanes, radii, simple_network, lambda_lasso)
    optimized_weights_with_lasso -= alpha * grad


non_zero_weights = np.sum(np.abs(optimized_weights_with_lasso) > 5e-3)
print(non_zero_weights)

print(optimized_weights_with_lasso)

# Parameters for gradient descent
alpha = 0.01
num_iterations = 800
lambda_l2 = 1

optimized_weights_with_lasso = np.ones(len(points)) / len(points)


gradient_descent_l2_loss(optimized_weights_with_lasso, points, hyperplanes, radii, simple_network, lambda_l2, alpha, num_iterations)

# Create a grid of points in the input space
grid_size = 50
x_range = np.linspace(0, 1, grid_size)
y_range = np.linspace(0, 1, grid_size)
X, Y = np.meshgrid(x_range, y_range)

# Compute the model output and optimized output for each point in the grid
model_output_grid = np.zeros((grid_size, grid_size))
optimized_output_grid = np.zeros((grid_size, grid_size))

for i, x in enumerate(x_range):
    for j, y in enumerate(y_range):
        point = np.array([x, y])
        model_output_grid[i, j] = model_function(point)
        optimized_output_grid[i, j] = sum([optimized_weights_with_lasso[k] * (np.dot(hyperplanes[k][0], point - points[k]) + hyperplanes[k][1]) if np.linalg.norm(point - points[k]) <= radii[k] else 0 for k in range(len(points))])

# Create a 3D plot of the model output and optimized output
fig = plt.figure(figsize=(16, 8))

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(X, Y, model_output_grid, cmap='viridis', alpha=0.8)
ax1.set_title("Model Output")
ax1.set_xlabel("X-axis")
ax1.set_ylabel("Y-axis")
ax1.set_zlabel("Z-axis")

ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(X, Y, optimized_output_grid, cmap='viridis', alpha=0.8)
ax2.set_title("Optimized Output")
ax2.set_xlabel("X-axis")
ax2.set_ylabel("Y-axis")
ax2.set_zlabel("Z-axis")

plt.show()





# Create a validation dataset
validation_data = [(np.random.uniform(-10, 10, 2), model_function(torch.tensor(np.random.uniform(-10, 10, 2))).detach().numpy()) for _ in range(1000)]

# Calculate the accuracy of the optimized weights without Lasso regularization
accuracy = calculate_accuracy(optimized_weights, hyperplanes, points, radii, validation_data)
print("Accuracy without Lasso regularization:", accuracy)

# Calculate the accuracy of the optimized weights with Lasso regularization
accuracy_lasso = calculate_accuracy(optimized_weights_with_lasso, hyperplanes, points, radii, validation_data)
print("Accuracy with Lasso regularization:", accuracy_lasso)


# Visualize the optimized output with Lasso regularization
optimized_output_grid_lasso = np.zeros((grid_size, grid_size))

for i, x in enumerate(x_range):
    for j, y in enumerate(y_range):
        point = np.array([x, y])
        optimized_output_grid_lasso[i, j] = sum([optimized_weights_with_lasso[k] * (np.dot(hyperplanes[k][0], point - points[k]) + hyperplanes[k][1]) if np.linalg.norm(point - points[k]) <= radii[k] else 0 for k in range(len(points))])

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, optimized_output_grid_lasso, cmap='viridis', alpha=0.8)
ax.set_title("Optimized Output with Lasso regularization")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
plt.show()


# Count the number of non-zero weights
non_zero_weights = np.sum(optimized_weights_lasso != 0)
print("Number of non-zero weights:", non_zero_weights)

# Get the number of layers in the first layer of the network
first_layer_neurons = len(model.layer1)
print("Number of layers in the first layer of the network:", first_layer_neurons)

# Test the main conjecture
if non_zero_weights == first_layer_neurons:
    print("The main conjecture is true.")
else:
    print("The main conjecture is false.")


lambd = 0.01  # Regularization parameter
initial_weights = np.random.rand(len(points))  # Random initial weights
res = minimize(l1_regularized_objective_function, initial_weights, args=(hyperplanes, points, radii, lambd), method='BFGS')
optimized_weights = res.x

print("Optimized Weights:", optimized_weights)


# Validation dataset
validation_points = [np.random.rand(2) for _ in range(1000)]

# Evaluate the optimized weights on the validation dataset
true_output = [model_function(point) for point in validation_points]
optimized_output = [sum([optimized_weights[k] * (np.dot(hyperplanes[k][0], point - points[k]) + hyperplanes[k][1]) if np.linalg.norm(point - points[k]) <= radii[k] else 0 for k in range(len(points))]) for point in validation_points]

# Calculate the accuracy
accuracy = np.mean(np.isclose(true_output, optimized_output, rtol=1e-1, atol=1e-1))
print("Accuracy:", accuracy)

# Visualize the resulting surface in 3D space
# ... (reuse the previous 3D visualization code and update the optimized_output_grid) ...
